In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import seaborn as sns
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from tensorflow.keras.utils import plot_model
# import matplotlib package for plotting purpose
import matplotlib.pyplot as plt


In [ ]:
# Initialize image data generator with rescaling
# Normalization
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# initializing training and testing path
train_dir = "/content/drive/MyDrive/Facial Emotion Recognition/train/"
test_dir = "/content/drive/MyDrive/Facial Emotion Recognition/test/"

# define function to read and count the number of images into the path for each
# class/emotion
def count_exp(path, set_):
  dict_ = {}
  for expression in os.listdir(path):
    dir_ = path + expression
    dict_[expression] = len(os.listdir(dir_))
  df = pd.DataFrame(dict_, index=[set_])
  return df

# execute the function for counting the images within training set
train_count = count_exp(train_dir, 'train')
# execute the function for counting the images within test set
test_count = count_exp(test_dir, 'test')
# print number of images within each class of training set
print(train_count)
# print number of images within each class of test set
print(test_count)


In [ ]:
# plot a bar graph for all classes within training set
train_count.transpose().plot(kind='bar')
# plot a bar graph for all classes within test set
test_count.transpose().plot(kind='bar')


In [ ]:
# Process training data into bacthes of augmented data using the
# flow_from_directory function into keras API
train_generator = train_data_gen.flow_from_directory(
'/content/drive/MyDrive/Facial Emotion Recognition/train/',
target_size=(48, 48),
batch_size=64,
color_mode="grayscale",
class_mode='categorical')


In [ ]:
# Process test data into bacthes of augmented data using the
# flow_from_directory function into keras API
validation_generator = validation_data_gen.flow_from_directory(
'/content/drive/MyDrive/Facial Emotion Recognition/test',
target_size=(48, 48),
batch_size=64,
color_mode="grayscale",
class_mode='categorical')

In [ ]:
# create model structure
emotion_model = Sequential()
# Block 1
# convolutional layer 1
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# Block 2
# convolutional layer 2
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
# Block 3
# convolutional layer 3
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy',
optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

In [ ]:
emotion_model.summary()

In [ ]:
# Train the neural network/model
emotion_model_info = emotion_model.fit(

train_generator,
steps_per_epoch=28709 // 64,
epochs=5, #reduced epochs
validation_data=validation_generator,
validation_steps=7178 // 64)


In [ ]:
# Plot the accuracy, validation accuracy, loss and validation loss for training and
# test dataset
print(emotion_model_info.history.keys())
plt.figure(figsize=(14,5))
plt.subplot(1,2,2)
plt.plot(emotion_model_info.history['accuracy'])
plt.plot(emotion_model_info.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,1)
plt.plot(emotion_model_info.history['loss'])
plt.plot(emotion_model_info.history['val_loss'])
plt.title('model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig("modelloss _modelaccuracy.png")


In [ ]:
# Print accuracy for training set
train_loss, train_accu = emotion_model.evaluate(train_generator)
# Print accuracy for test set
test_loss, test_accu = emotion_model.evaluate(validation_generator)
print("final train accuracy = {:.2f}, validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))


In [ ]:
# write model structure in json file
model_json = emotion_model.to_json()
# use the write method to write our json weights
# using with loop to write the file
with open("emotion_model.json", "w") as json_file:
  # write the json file
  json_file.write(model_json)

In [ ]:
# save trained model weight and the modek in .h5 file
emotion_model.save_weights('model_file_30epochs.h5')
# emotion_model.save("model.h5")


In [ ]:
 # Confusion Matrix and Classification on training set
y_pred = emotion_model.predict(train_generator)
y_pred = np.argmax(y_pred, axis=1)
class_labels = validation_generator.class_indices
class_labels = {v:k for k,v in class_labels.items()}

from sklearn.metrics import classification_report, confusion_matrix
cm_train = confusion_matrix(train_generator.classes, y_pred)
print('Confusion Matrix')
print(cm_train)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(train_generator.classes, y_pred,
target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_train, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)
plt.savefig("confusion_matrix_train.png")


In [ ]:
# Confusion Matrix and Classification on test set
y_pred = emotion_model.predict(validation_generator)
y_pred = np.argmax(y_pred, axis=1)
class_labels = validation_generator.class_indices
class_labels = {v:k for k,v in class_labels.items()}

#from sklearn.metrics import classification_report, confusion_matrix
cm_test = confusion_matrix(validation_generator.classes, y_pred)
print('Confusion Matrix')
print(cm_test)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred,
target_names=target_names))


plt.figure(figsize=(8,8))
plt.imshow(cm_test, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)
plt.savefig("confusion_matrix_test.png")